In [2]:
import torch
print(torch.cuda.is_available())  # Should return True if CUDA is available
print(torch.version.cuda)         # Should print your CUDA version, e.g


True
12.1


In [3]:
import matplotlib.pyplot as plt
import networkx as nx
import os
from pathlib import Path
import pandas as pd
import seaborn as sns
import torch
from torch_geometric.data import Data
from torch_geometric import utils
from environment_manager import *
name = "240318-initial parsing"
env = load_create_environment(name)
GHComponentTable.initialise()

Setting environment variables
Copying vanilla components
File copied successfully from C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\Grasshopper Components\240318-VanillaComponents\vanilla_components.csv to ExtractionEnvironments\240318-initial parsing\00-VanillaComponents\vanilla_components.csv.
Copying components
Copying gh files


In [4]:
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.colors as mcolors

def plot_graph_by_category(graph):
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 8))

    # Get the unique categories
    categories = set(nx.get_node_attributes(graph, 'category').values())

    # Create a color map based on the number of categories
    num_colors = len(categories)
    color_map = plt.cm.get_cmap('viridis', num_colors)

    # Create a dictionary mapping categories to colors
    category_colors = {}
    for i, category in enumerate(categories):
        category_colors[category] = mcolors.to_hex(color_map(i))

    # Create a dictionary of node positions
    pos = nx.get_node_attributes(graph, 'pos')
    if not pos:  # If 'pos' attribute is not available, use 'x' and 'y' attributes
        pos = {node_id: (data['x'], data['y']) for node_id, data in graph.nodes(data=True)}

    # Create a list of colors based on the node categories
    node_colors = [category_colors[graph.nodes[node].get('category', '')] for node in graph.nodes()]

    # Draw the graph
    nx.draw(graph, pos=pos, with_labels=False, node_size=10, node_color=node_colors, edge_color='gray', arrows=True, ax=ax)

    # Create a legend
    legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=category,
                                  markerfacecolor=color, markersize=8)
                       for category, color in category_colors.items()]
    ax.legend(handles=legend_elements, title='Categories', loc='upper right')

    # Show the plot
    plt.axis('equal')
    plt.show()

In [5]:
import os
import networkx as nx

graph_folder = r'C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\ExtractionEnvironments\240318-initial parsing\05-GraphML'
# Initialize an empty list to store dictionaries
data_list = []


# Create a new graph to store the combined graph
combined_graph = nx.DiGraph()
graph_data = []
for i, file in enumerate(os.listdir(graph_folder)):
    if file.endswith('.graphml'):
        G = nx.read_graphml(os.path.join(graph_folder, file))
        # Skip files with no nodes to avoid division by zero
        if G.number_of_nodes() < 5:
            continue
        if G.number_of_edges() == 0:
            continue
        graph_data.append(G)
        combined_graph.add_nodes_from(G.nodes(data=True))
        combined_graph.add_edges_from(G.edges(data=True))

    

In [21]:
print(graph_data[0].nodes(data=True))


[('c9cde350-c7de-4df6-8a70-82cc57fd78ac', {'x': 428.0, 'y': 193.0, 'compid': '807b86e3-be8d-4970-92b5-f8cdcb45b06b', 'category': 'Curve'}), ('8f48460a-8768-456a-a63e-e1183c8c2f9e', {'x': 591.0, 'y': 217.0, 'compid': '962034e9-cc27-4394-afc4-5c16e3447cf9', 'category': 'Surface'}), ('1761219f-05c6-480d-a4f6-adfa5b5c8ea5', {'x': 139.5, 'y': 201.0, 'compid': '57da07bd-ecab-415d-9d86-af36d7073abc', 'category': 'Params'}), ('bb87905f-0d55-44fe-aea0-c50778c90631', {'x': 763.0, 'y': 250.0, 'compid': '5106bafc-d5d4-4983-83e7-7be3ed07f502', 'category': 'Surface'}), ('8b71cf8a-5e29-4ea0-8efe-04b349b8ad46', {'x': 435.0, 'y': 267.0, 'compid': '9103c240-a6a9-4223-9b42-dbd19bf38e2b', 'category': 'Vector'}), ('1087a0c7-90b1-41cd-88ed-22b3ec5cac33', {'x': 176.3834991455078, 'y': 271.78900146484375, 'compid': '57da07bd-ecab-415d-9d86-af36d7073abc', 'category': 'Params'}), ('f1c9c47f-4fd6-4aa7-911e-301230895ad6', {'x': 892.0, 'y': 215.0, 'compid': '41aa4112-9c9b-42f4-847e-503b9d90e4c7', 'category': 'Sets

In [22]:
for node, data in combined_graph.nodes(data=True):
    # Assign the values of 'x' and 'y' to 'posx' and 'posy'
    data['posx'] = data['x']
    data['posy'] = data['y']
    # Optionally, remove the old 'x' and 'y' keys if they're no longer needed
    del data['x']
    del data['y']

In [24]:
d = utils.from_networkx(combined_graph)
d

Data(edge_index=[2, 132214], compid=[106450], category=[106450], posx=[106450], posy=[106450], distance=[132214], s_paramidx=[132214], r_paramidx=[132214], s_access=[132214], r_access=[132214], s_datamapping=[132214], r_datamapping=[132214], s_is_optional=[132214], r_is_optional=[132214], num_nodes=106450)

In [8]:
# # Create a lookup table for all the nodes
# """we need to reorder the graphs because the node indices are not unique across the graphs and pytorch will order their indicies according to each node index"""
# node_lookup = {}
# node_counter = 0
# for i, graph in enumerate(graph_data):
#     for node in graph.nodes():
#         if node not in node_lookup:
#             node_lookup[node] = node_counter
#             node_counter += 1
# 
# reordered_graphs = []
# for graph in graph_data:
#     reordered_graph = nx.relabel_nodes(graph, {node: node_lookup[node] for node in graph.nodes()})
#     reordered_graphs.append(reordered_graph)



In [18]:
#remove the isolated nodes from the combined graph
combined_graph.remove_nodes_from(list(nx.isolates(combined_graph)))
combined_graph
# ds = utils.from_networkx(combined_graph)
# print(f'Number of nodes: {d.num_nodes}')
# print(f'Number of edges: {d.num_edges}')
# print(f'Average node degree: {d.num_edges / d.num_nodes:.2f}')
# print(f'Contains isolated nodes: {d.contains_isolated_nodes()}')
# print(f'Contains self-loops: {d.contains_self_loops()}')
# print(f'Is undirected: {d.is_undirected()}')


KeyError: 'x'

In [10]:
degs = sorted([d for n, d in list(nx.degree(combined_graph))], reverse=True)
print(f'Highest node degree: {degs[0]}')
print(f'Lowest node degree: {degs[-1]}')
print(f'Median node degree: {degs[len(degs) // 2]}')
print(f'Mean node degree: {sum(degs) / len(degs):.2f}')

Highest node degree: 53
Lowest node degree: 1
Median node degree: 2
Mean node degree: 2.48


In [17]:
ds

Data(x=[106450], edge_index=[2, 132214], y=[106450], compid=[106450], category=[106450], distance=[132214], s_paramidx=[132214], r_paramidx=[132214], s_access=[132214], r_access=[132214], s_datamapping=[132214], r_datamapping=[132214], s_is_optional=[132214], r_is_optional=[132214])

Implemending the GNN model

In [12]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, LayerNorm, Sequential, Linear

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(ds.num_features, 16)
        self.conv2 = GCNConv(16, 16)
        self.conv3 = GCNConv(16, 16)
        self.lns = LayerNorm(16)
        self.post_mp = torch.nn.Sequential(
            Linear(16, 16),
            torch.nn.Dropout(0.25),
            Linear(16, 2)
        )
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.25, training=self.training)
                # x= self.conv2(x, edge_index)
                # x = F.relu(x)
                # x = F.dropout(x, p=0.25, training=self.training)
                # x =self.lns(x)
                # x = self.conv3(x, edge_index)
                # x= F.relu(x)
                # x = F.dropout(x, p=0.25, training=self.training)
                # x = self.post_mp(x)
        return x, F.log_softmax(x, dim=1)
    


In [13]:
from sklearn.manifold import TSNE
model = GNN()
embedding, out = model(ds)
color_list = ["red", "orange", "green", "blue", "purple", "brown", "pink", "gray", "olive", "cyan", "lime"] #C
colors = [] #D
colors += [color_list[y] for y in ds.y.detach().numpy()] #E
xs, ys = zip(*TSNE().fit_transform(embedding.detach().numpy())) #F
plt.scatter(xs, ys, c=colors) #G

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)

In [ ]:
# combined_graph = nx.compose_all(graph_data[:2])
# for graph in graph_data[:2]:
#     plot_graph_by_category(graph)
# plot_graph_by_category(combined_graph)

In [ ]:
print(len(combined_graph))
print(combined_graph.number_of_nodes())
print(nx.number_connected_components(combined_graph))


In [ ]:
data = [utils.from_networkx(G) for G in graph_data]
data[0]['edge_index']

In [ ]:
# import System
# # import Counter 
# from collections import Counter

# guids = [v for d in data for v in d['guid']]
# counter = Counter(guids)
# tops = counter.most_common(1000)
# names = []
# categories = []
# counts = []
# for t, c in tops:
#     name = GHComponentTable.search_component_by_guid(System.Guid(t)).Desc.Name
#     names.append(name)
#     category = GHComponentTable.search_component_by_guid(System.Guid(t)).Desc.Category
#     categories.append(category)
#     counts.append(c)
# # components = [GHComponentTable.search_component_by_guid(System.Guid(g)) for g in guids]
# # components

In [ ]:
plot = False
if plot:
    # Set the font size globally
    plt.rcParams.update({'font.size': 5})  
    plt.figure(figsize=(10,200))
    
    x = counts
    y = [f"{categories[i]}:{name}" for i, name in enumerate(names)]
    sns.barplot(x=x, y=y)
